# Part 2: HLS Kernel Development

## 1. Baseline code

First, we have a piece of Baseline's code as shown below. The basic functions of the `fir_wrap` function and the `fir` function. Here we focus more on the process of implementing their functions. In the `fir` function, each loop has two basic operations. First, it multiplies and accumulates the current input signal, puts the result in `acc`, and moves the elements of the `shift_reg` array forward. At the end of the loop, the `acc` variable stores the result of the current filtered signal, giving the final output. But within this function, there is a large amount of control logic present and it is executed serially.

```cpp
#include "./fir.h"

coef_t c[N];

void fir(data_t *y, data_t x)
{
    static data_t shift_reg[N];
    acc_t acc;
    int i;
    acc = 0;
    for (i = N - 1; i >= 0; i--)
    {
        if (i == 0)
        {
            acc += x * c[0];
            shift_reg[0] = x;
        }
        else
        {
            shift_reg[i] = shift_reg[i - 1];
            acc += shift_reg[i] * c[i];
        }
    }
    *y = acc;
}

void fir_wrap(data_t *y, data_t *x, int len, coef_t *coef)
{
	data_t res;

	for (int i =0; i < N; i++)
	{
		c[i] = *coef++;
	}

	for (int i = 0; i < len; i++)
	{
		fir(&res,*x++);
		*y = res;
		y++;
	}
}
```

In Vitis HLS software, we can perform a synthesis to see its Latency and resource usage. We can see that the Latency of the FIR IP without any optimization is 20607 and the resource usage is less.

<img src="./image/HLS_baseline.png" alt="HLS_baseline.png"  style="zoom:70%;" />

We ran it on the PYNQ board and got Baseline to run in about 3.67 seconds. This is an 8x delay over running it on Python alone, but we can improve it with subsequent optimizations.

<img src="./image/Time_baseline.png" alt="Time_baseline.png"  style="zoom:70%;" />

Next, we propose some optimization methods to improve its performance.

## 2 Optimization from Micro Perspective

## 2.1 Code hoisting to remove unnecessary conditional operations

In the first step, there is an `if-else` branch statement in the `for` loop of the `FIR` function. At each loop, HLS generates a logic circuit to perform the `if-else` statement check to determine if the condition is met. So the `if-else` branch statement in the `for` loop is inefficient and we can optimize it. We can remove the `if-else` branch statement in the `for` loop while ensuring correct functionality. We can see that the `if` statement will be satisfied only at the last loop, so we can execute the conditional statement at the end of the loop. The modified code is shown below. This code structure will make it easier for us to optimize later.

```cpp
for (i = N - 1; i > 0; i--)
{
    shift_reg[i] = shift_reg[i - 1];
    acc += shift_reg[i] * c[i];
}

acc += x * c[0];
shift_reg[0] = x;
```

By removing the `if-else` branch, the l atency is reduced to about half of its original size, and performance is greatly improved. The hardware on the slice is roughly three times more utilized than before.

<img src="./image/HLS_ifelse.png" alt="HLS_ifelse.png"  style="zoom:70%;" />

We ran the code without the `if-else` branch on the PYNQ board and found that the time dropped from 3.67 seconds to 1.93 seconds.

<img src="./image/Time_ifelse.png" alt="Time_ifelse.png"  style="zoom:70%;" />

## 2.2 Array partitioning to enable loop parallelism

We can also increase the amount of data accessed by certain compilation directives. For example, `c[]` and `shift_reg[]` arrays, they have only one read/write port each time, which makes the amount of data access limited each time, and can only get an equal amount of data by increasing the number of accesses, which will inevitably increase the Latency. write multiple values at the same time. The code is shown in the following figure. `variable` refers to the variable name of the array to be partitioned, and `complete` means that each element of the array will be partitioned into independent units, so that each element can be read and written independently of each other, without interference, to pave the way for the optimization later. Of course, you can also replace `complete` with others, such as `block` and `cyclic`. `block` means splitting the array horizontally, `cyclic` means splitting the array vertically, and the keyword `factor=<int>` means splitting into several groups of data.

<img src="./image/array_partition.png" alt="array_partition.png"  style="zoom:30%;" />

In the code, we represent the directives as follows.

```cpp
coef_t c[N];

void fir(data_t *y, data_t x)
{
#pragma HLS ARRAY_PARTITION variable=c complete dim=0
    static data_t shift_reg[N];
#pragma HLS ARRAY_PARTITION variable=shift_reg complete dim=0
    ......   
}
```

After adding the above compilation directive, we can see that the latency does not change much in the HLS synthesis report, because we have not added the subsequent operation steps. The resources of FF and LUT are significantly increased due to the array partitioning.

<img src="./image/HLS_array.png" alt="HLS_array.png"  style="zoom:70%;" />

### 2.3 Loop split for tailored optimizations

In the `Baseline` module, we mentioned that each loop has two basic operations, which are multiply-accumulate operation and array shift operation. We can split this one loop into two loops and perform different optimization operations for different loop characteristics. Here is the code after splitting the loop, TDL is the Time Delay line, and MAC is the multiply-accumulate operation. Naming the loops in this way allows us to better analyze them in the synthesis report.

```cpp
TDL:
for(i = N-1; i > 0; i--){
	shift_reg[i] = shift_reg[i-1];
}
shift_reg[0] = x;

MAC:
for(i = N-1;i >= 0;i--){
	acc += shift_reg[i] * c[i];
}
```

Of course, fissing the loop alone does not provide any substantial optimization for Latency and resources, and we have to add subsequent optimizations to achieve a 1+1>2 effect. Here, we do not show the hardware resource utilization and runtime after loop fission. Because this part does not improve the hardware structure, it is mainly in the preparation for the later optimization.

### 2.4 Loop Unrolling & Loop Pipelining

Our HLS tool will synthesize the for loop into a sequential execution by default. We can change this to parallel computation to improve the efficiency of operations. Unroll means creating multiple independent operations instead of a single set of operations. Obviously then, the use of resources must increase dramatically. Let's use a loop as an example. As shown in the figure below, if a for loop needs to loop 3 times, in the non-parallel case, it needs to execute read, compute, and write 3 times in a sequential loop. 1 ALU (e.g., adder, multiplier, divider, etc.) can be reused 3 times, spending a total of 9 cycles. In the case of full parallel expansion, 3 reads, calculations, and writes can be performed simultaneously. In this case, 1 ALU is not sufficient for 3 calculations at the same time, and 3 ALUs are needed to complete the same amount of operations for a total of 3 cycles.

<img src="./image/The_operation_process_of_pipeline.png" alt="The_operation_process_of_unroll.png"  style="zoom:100%;" />

In the `TDL` loop, when we add `#pragma HLS unroll`, the compiler expands all this code, i.e., performs all shift operations in one loop iteration, greatly reducing the number of iterations. This also requires that we have N-2 accesses to the `shift_reg` array. This is where the array splitting we laid out earlier comes in handy. We go ahead and fiss the `shift_reg` array into separate registers, making it possible to read and write to each register each cycle.

```cpp
TDL:
    for (i = N - 1; i > 0; i--)
    {
#pragma HLS unroll 
        shift_reg[i] = shift_reg[i - 1];
    }
    shift_reg[0] = x;
```

The hardware design comparison diagram is shown below. In a non-parallel loop, we only have limited resources, so we need to wait until the previous operation is done before proceeding to the next one. In a parallel loop, we can do the same operation at the same time and save time.

<img src="./image/TDL_unroll_hardware.png" alt="TDL_unroll_hardware.png"  style="zoom:40%;" />

In the MAC loop, multiplication and addition are performed once per iteration and the result of the multiplication is accumulated into `acc`. We manually expand the MAC loop with an expansion factor of 4, which also eliminates the accumulation dependency each time. Also after the MAC loop is expanded, there is an additive for loop, which is important to execute the rest of the process in case the number of iterations is not an unfolding factor. This is the same effect we achieve by adding `pragma`.

```CPP
 MAC:
	for (i = N - 1; i >= 3; i-=4)
	{
        acc += shift_reg[i] * c[i] + shift_reg[i-1] * c[i-1] + shift_reg[i-2] * c[i-2] + shift_reg[i-3] * c[i-3];
	}
	for(; i >= 0; i--){
    	acc += shift_reg[i] * c[i];
    }
```

Another common approach is the pipeline operation. The concept of pipeline in a factory is similar in that each production unit focuses on a particular segment of work to increase efficiency and productivity. Pipelining is a quasi-parallel processing implementation technique in which multiple instructions overlap during program execution. It reduces the initiation interval (II) for a function or loop by allowing the concurrent execution of operations,measured in clock cycles. Pipeline does not add as much to resources as unroll, but it essentially makes resources less vacant.

<img src="./image/The_operation_process_of_pipeline.png" alt="The_operation_process_of_pipeline.png"  style="zoom:100%;" />

Consider the MAC loop. In the MAC loop, we need to perform four operations, which are read `c[]`, read `shift_reg[]`, multiply `c[]` and `shift_reg[]`, and accumulate operation. There is no dependency between the two array reads, but it takes one cycle to provide the memory address and another cycle to complete the data transfer, so it still takes two cycles to complete. The multiply and accumulate operations have dependencies and take two cycles to complete. Therefore, it takes four cycles to complete a MAC. Then with unpipelined operation, it takes 4N cycles to complete the MAC cycle. When we pipelined, the second iteration starts from the second cycle and the third drop starts from the third cycle, and it takes N+3 cycles to complete N iterations. Also II indicates the number of clock cycles from this loop to the start of the next loop in case there are dependencies or resource constraints between internal operations. We can adjust the value of II manually, or let the HLS tool tell you the most optimal value that can be achieved.

```cpp
MAC:
	for(i = N-1;i >= 0;i--){
#pragma HLS PIPELINE II=1
		acc += shift_reg[i] * c[i];
	}
```

Its hardware design is compared in the following diagram. In the pipeline loop, we optimize multiple iterative operations to overlap.

<img src="./image/MAC_pipeline_hardware.png" alt="MAC_pipeline_hardware"  style="zoom:40%;" />

After the optimization of loop unfolding and loop flow, Latency is reduced from more than 10,000 to more than 200, which improves the performance by nearly 50 times. The utilization of the computing hardware DSP and the storage lookup hardware FF and LUT is also dramatically increased. Although the DSP may exceed the resources already available on the PYNQ board in the HLS synthesis report, it is automatically partially optimized to work properly on PYNQ in the subsequent tool chain.

<img src="./image/HLS_unrollpipeline.png" alt="HLS_unrollpipeline.png"  style="zoom:70%;" />

After unroll and pipeline, we ran the final code on the PYNQ board and the run time went from 1.88 seconds to 0.07 seconds, achieving a faster speed than the software.

<img src="./image/Time_unrollpipeline.png" alt="Time_unrollpipeline.png"  style="zoom:70%;" />

## 3. Putting it All Together -- Kernel Code Analysis

Before we start the implementation of FIR accelerated IP, we should get familiar with the High Level Synthesis (HLS) design flow. Here is the official github documentation https://github.com/Xilinx/xup_high_level_synthesis_design_flow. You can find the steps and tutorials for using the toolchain in there.

In this section, we will show the concrete code of the hardware implementation and roughly talk about the specific implementation principles. Here is the code for the implementation of the FIR accelerated IP.

```cpp
#include "./fir.h"

coef_t c[N];

void fir(data_t *y, data_t x)
{

#pragma HLS ARRAY_PARTITION variable=c complete dim=0
    static data_t shift_reg[N];
#pragma HLS ARRAY_PARTITION variable=shift_reg complete dim=0
    acc_t acc;
    int i;
    acc = 0;
//#pragma HLS PIPELINE II=1
    TDL:
    for(i = N-1; i > 0; i--){
#pragma HLS unroll
    	shift_reg[i] = shift_reg[i-1];
    }
    shift_reg[0] = x;

    MAC:
    for(i = N-1;i >= 0;i--){
#pragma HLS unroll
    	acc += shift_reg[i] * c[i];
    }

    *y = acc;
}

void fir_wrap(data_t *y, data_t *x, int len, coef_t *coef)
{
#pragma HLS INTERFACE m_axi depth=100 port=y
#pragma HLS INTERFACE m_axi depth=100 port=x
#pragma HLS INTERFACE m_axi depth=99 port=coef
#pragma HLS INTERFACE s_axilite port=len  bundle=CTRL
#pragma HLS INTERFACE s_axilite port=return bundle=CTRL

	data_t res;

	for (int i =0; i < N; i++)
	{
#pragma HLS PIPELINE II=1
		c[i] = *coef++;
	}

	for (int i = 0; i < len; i++)
	{
#pragma HLS LOOP_TRIPCOUNT  max=100 min=100
#pragma HLS PIPELINE II=1
		fir(&res,*x++);
		*y = res;
		y++;
	}

}
```

The `fir` function achieves the function of completing the filtered signal at a certain point in time. The input of the function is the original signal, and the output is the filtered signal. Its parameters are mainly fixed tap coefficients, and the multiplication and accumulation operations are performed by looping. The main function of the `fir_wrap` function is to localize the tap coefficients `coef[]` and to call the fir function through a loop to calculate the filtered signal at each time point.

In addition to the familiar Cpp language, we also include macros like `#pragma` in some parts of the code to instruct the compiler to perform some specific actions, such as parallelism, pipelining, etc., to achieve hardware acceleration. This part will be discussed in detail later. At this point, the implementation code, together with the HLS compilation instructions, makes up the main code for hardware acceleration.

## **Stretch goals**

1. Please use the basic knowledge of HLS above to implement an accelerated core for FFT.
2. After completing the IP design, we are about to run this IP on PYNQ, so how is it connected to the hardware? And how is the specific call of the software hardware implemented?

<p align="center">Copyright© 2024 Advanced Micro Devices</p>